# Kernel Semántico

En este ejemplo de código, utilizarás el marco de inteligencia artificial [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) para crear un agente básico.

El objetivo de este ejemplo es mostrarte los pasos que utilizaremos más adelante en los ejemplos adicionales de código al implementar los diferentes patrones de agentes.


## Importar los paquetes de Python necesarios


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## Creando el Cliente

En este ejemplo, utilizaremos [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) para acceder al LLM.

El `ai_model_id` está definido como `gpt-4o-mini`. Prueba cambiar el modelo a otro disponible en el mercado de GitHub Models para observar diferentes resultados.

Para que podamos usar el `Azure Inference SDK`, que se utiliza para el `base_url` de GitHub Models, emplearemos el conector `OpenAIChatCompletion` dentro de Semantic Kernel. También hay otros [conectores disponibles](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) para usar Semantic Kernel con otros proveedores de modelos.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Creando el Agente

A continuación, creamos el Agente llamado `TravelAgent`.

Para este ejemplo, estamos utilizando instrucciones muy simples. Puedes cambiar estas instrucciones para ver cómo responde el agente de manera diferente.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## Ejecutar el Agente

Ahora podemos ejecutar el Agente definiendo un hilo del tipo `ChatHistoryAgentThread`. Cualquier mensaje del sistema necesario se proporciona al argumento de palabra clave `messages` de invoke_stream del agente.

Una vez definidos, creamos un `user_inputs`, que será lo que el usuario envíe al agente. En este caso, hemos configurado este mensaje como `Planéame unas vacaciones soleadas`.

Siéntete libre de cambiar este mensaje para ver cómo responde el agente de manera diferente.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Si bien nos esforzamos por garantizar la precisión, tenga en cuenta que las traducciones automatizadas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
